# Gen Retriever

## Einführung


In [1]:
import os

# Aktuelles Arbeitsverzeichnis ermitteln
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

In [2]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("duckdb:///data/magpie.db")

db = SQLDatabase.from_uri("duckdb:///data/drittmittel_hs.db")
db.get_table_names()
print(db.get_table_info(["datensatz_drittmittel_hochschule"]))

c:\Users\Hueck\miniconda3\envs\RAG_retriever\lib\site-packages\duckdb_engine\__init__.py:174: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(



CREATE TABLE datensatz_drittmittel_hochschule (
	jahr INTEGER, 
	id INTEGER, 
	"Variable" VARCHAR, 
	"Zeit" TIMESTAMP WITHOUT TIME ZONE, 
	"Hochschule" VARCHAR, 
	"Wert" NUMERIC(18, 3), 
	"Einheit" VARCHAR, 
	"Quelle" VARCHAR
)

/*
3 rows from datensatz_drittmittel_hochschule table:
jahr	id	Variable	Zeit	Hochschule	Wert	Einheit	Quelle
2006	30746	Drittmittel vom Bund	2006-01-01 00:00:00	Universität Kassel	3966.000	in Tsd. Euro	Destatis (Sonderauswertung)
2007	30747	Drittmittel vom Bund	2007-01-01 00:00:00	Universität Kassel	6274.000	in Tsd. Euro	Destatis (Sonderauswertung)
2008	30748	Drittmittel vom Bund	2008-01-01 00:00:00	Universität Kassel	5980.000	in Tsd. Euro	Destatis (Sonderauswertung)
*/


C:\Users\Hueck\AppData\Local\Temp\ipykernel_36004\2876354156.py:5: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 1.0. Use :meth:`~get_usable_table_names` instead.
  db.get_table_names()


In [3]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


Variable = query_as_list(db, "SELECT Variable FROM datensatz_drittmittel_hochschule ")
Hochschule = query_as_list(db, "SELECT Hochschule FROM datensatz_drittmittel_hochschule ")

print(Hochschule)

['Philosophisch-Theologische Hochschule', 'Georg-August-Universität Göttingen (ohne Klinikum)', 'GA Hochschule der digitalen Gesellschaft', 'Fachhochschule für Forstwirtschaft', 'Pädagogische Hochschule Heidelberg', 'Fachhochschule für Interkulturelle Theologie', 'Fachhochschule Oldenburg/Ostfriesland/Wilhelmshaven', 'Fachhochschule Erfurt', 'Kunsthochschule für Medien', 'Julius-Maximilians-Universität Würzburg (ohne Klinikum)', 'Fachhochschule im Deutschen Roten Kreuz', 'Universität Duisburg-Essen  (Klinikum)', 'Hochschule für Musik Freiburg i. Br.', 'Hochschule Kaiserslautern', 'Jade Hochschule Wilhelmshaven/Oldenburg/Elsfleth', 'Hochschule Ludwigsburg für öffentliche Verwaltung und Finanzen', 'Ruhr-Universität Bochum (Klinikum)', 'Hochschule für Gesundheitsorientierte Wissenschaften Rhein-Neckar (HGWR)', 'Hochschule der Sparkassen-Finanzgruppe University of Applied Sciences - Bonn', 'Universitätsklinikum Gießen und Marburg, Abt. Gießen', 'Universität Greifswald Medizinische Fakultät

Mit dieser Funktion können wir ein Retriever-Tool erstellen, das der Bot nach eigenem Ermessen ausführen kann.

Wählen wir für diesen Schritt ein Embeddingmodel und einen Vektorspeicher aus:

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch

# Überprüfen, ob eine GPU verfügbar ist
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(torch.cuda.get_device_name(0))
# Modell laden und auf die GPU verschieben
# Specify the device in model_kwargs
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base", 
    model_kwargs={"device": device}
)

# Chroma Vektorspeicher initialisieren
vector_store = Chroma(embedding_function=embeddings)

Using device: cuda
NVIDIA GeForce RTX 4060 Ti


C:\Users\Hueck\AppData\Local\Temp\ipykernel_36004\2039575610.py:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embeddings)


Wir können nun ein Suchwerkzeug konstruieren, das die relevanten Eigennamen in der Datenbank durchsucht:

In [8]:
from langchain.agents.agent_toolkits import create_retriever_tool

_ = vector_store.add_texts(Variable + Hochschule)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
description = (
    "Use to find proper nouns and their correct spellings. Input is an approximate spelling "
    "of the proper noun, output is the closest valid proper noun. Use the noun most similar to the search."
)
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [11]:
# _ = vector_store.add_texts(
#     ["Freie Universität Berlin"],  # Hauptbegriff
#     [{"synonyms": "FU Berlin; Freie Univ. Berlin"}]  # Synonyme als Metadaten
# )

In [34]:
print(retriever_tool.invoke("Wie hoch waren die Drittmittel von Gemeinden und Zweckverbänden der Uni Kassel im Jahr 2008?"))

Drittmittel von Gemeinden und Zweckverbänden

Drittmittel von Stiftungen

Drittmittel von Hochschulfördergesellschaften

Drittmittel vom Bund

Drittmittel von der Bundesanstalt für Arbeit


In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_sql_query_chain

def get_sql_chain(llm, db, table_info, retriever_tool, top_k=10):
    template = f"""Given an input question, first create a syntactically
    correct SQL query to run in {db.dialect}, then look at the results of the
    query and return the answer to the input question.

    Unless otherwise specified, do not return more than {{top_k}} rows.

    Never query for all columns from a table. You must query only the
    columns that are needed to answer the question. Wrap each column name
    in double quotes (") to denote them as delimited identifiers.

    Pay attention to use only the column names present in the tables
    below. Be careful to not query for columns that do not exist. Also, pay
    attention to which column is in which table. Query only the columns you
    need to answer the question.

    If a filter value or variable category is required, include it in the 
    WHERE clause, and combine multiple filter criteria (if applicable) 
    using AND or OR as appropriate. Use the retriever_tool.

    Here is the schema for the database:
    {{table_info}}

    Additional info: {{input}}

    Return only the SQL query such that your response could be copied
    verbatim into the SQL terminal.
    """

    prompt = PromptTemplate.from_template(template)

    def validate_values(question, retriever_tool):
        """
        Validate and enrich the input question by using the retriever tool
        to check for valid variable values or categories.
        """
        extracted_values = retriever_tool.run(question)
        if extracted_values:
            print("Validated values:", extracted_values)
        return extracted_values or question  # Fallback to original question if no match is found

    sql_chain = create_sql_query_chain(llm, db, prompt)

    return sql_chain, validate_values

def natural_language_chain(question, llm, db, retriever_tool):
    table_info = db.get_table_info()
    sql_chain, validate_values = get_sql_chain(llm, db, table_info=table_info, retriever_tool=retriever_tool)

    # Validate or enrich the question with proper variable values
    validated_question = validate_values(question, retriever_tool)

    template = f"""
        You are a chatbot named Sparklehorse created. Based on the table schema given below, the SQL query and the SQL response, enter an answer
        that corresponds exactly to the language of the user's question. Think carefully and make sure that your answer is precise and easy to understand.

        SQL Query: {{query}}
        User question: {{question}}
        SQL Response: {{response}}
        """

    prompt = PromptTemplate.from_template(template)

    # Create the intermediate chain to extract SQL query
    intermediate_chain = RunnablePassthrough.assign(query=sql_chain)

    # Get the SQL query
    intermediate_result = intermediate_chain.invoke({"question": validated_question})
    sql_query = intermediate_result["query"]

    # Debug: Print the SQL query
    print("Generated SQL Query for Debugging:")
    print(sql_query)

    # Continue with the full chain execution
    chain = (
        intermediate_chain.assign(
            response=itemgetter("query") | QuerySQLDataBaseTool(db=db)
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    response = chain.invoke({"question": validated_question})

    print(response)

    return response

In [31]:
from langchain_ollama import ChatOllama
import re

llm = ChatOllama(
    model="llama3.1:8b-instruct-q4_0",
    temperature=0,
    server_url="http://127.0.0.1:11434",
)

In [32]:
_ = natural_language_chain('Wie hoch waren die Drittmittel von Gemeinden der Uni Kassel nur im Jahr 2010?', llm, db,retriever_tool )

Validated values: Drittmittel von Gemeinden und Zweckverbänden

Drittmittel von Hochschulfördergesellschaften

Drittmittel von Stiftungen

Universität Kassel

Drittmittel vom Bund
Generated SQL Query for Debugging:
To create a syntactically correct SQL query for duckdb, I'll need more information about the input question you'd like to answer. However, based on the provided schema and additional info, here's an example of how we might craft a query:

Let's say your question is: "What were the Drittmittel from Hochschulfördergesellschaften for Universität Kassel in 2007?"

Here's the SQL query to answer this question:
```sql
SELECT 
    "Variable", 
    "Wert" 
FROM 
    datensatz_drittmittel_hochschule 
WHERE 
    "Hochschule" = 'Universität Kassel' AND 
    "Zeit" LIKE '%2007%' AND 
    "Variable" = 'Drittmittel von Hochschulfördergesellschaften';
```
This query selects the `Variable` and `Wert` columns from the `datensatz_drittmittel_hochschule` table where the `Hochschule` is 'Univer

In [25]:
result = SQLDatabase.from_uri("duckdb:///data/drittmittel_hs.db").run("""
    SELECT "Hochschule", "Wert"
    FROM datensatz_drittmittel_hochschule
    WHERE "Variable" = 'Drittmittel vom Bund'
      AND "Hochschule" = 'Universität Kassel'
    ORDER BY "Wert" DESC
    LIMIT 5;
""")

In [26]:
print(result)

[('Universität Kassel', Decimal('27493.000')), ('Universität Kassel', Decimal('27260.000')), ('Universität Kassel', Decimal('25835.000')), ('Universität Kassel', Decimal('25689.000')), ('Universität Kassel', Decimal('25281.000'))]
